In [2]:
import numpy as np 
import pandas as pd 
import geopandas as gpd 

In [4]:
# B302201: Time leaving home (5) by Means of transportation (7)
flow_df = pd.read_csv('traffic_inputs/06_ctpp_ca/CA_2012thru2016_B302201.csv')
display(flow_df.head())
car_flow_df = flow_df.loc[flow_df['LINENO'].isin([16, 17, 23, 24])]
tract_df = car_flow_df[car_flow_df.GEOID.str.contains(r'C5400US06\d{9}06\d{9}')].reset_index(drop=True)
tract_df.to_csv('ctpp/CA_2012thru2016_B302201_C5400US.csv', index=False)

,GEOID,TBLID,LINENO,EST,MOE,SOURCE
0,C4200US0601,B302201,1,155,+/-87,NaN
1,C4200US0601,B302201,2,75,+/-56,NaN
2,C4200US0601,B302201,3,25,+/-38,NaN
3,C4200US0601,B302201,5,15,+/-21,NaN
4,C4200US0601,B302201,6,45,+/-50,NaN


In [9]:
# show state, county and tract code separately
tract_df = pd.read_csv('ctpp/CA_2012thru2016_B302201_C5400US.csv')
tract_df['origin_state'] = tract_df.GEOID.str[7:9]
tract_df['origin_county'] = tract_df.GEOID.str[9:12]
tract_df['origin_tract'] = tract_df.GEOID.str[12:18]
tract_df['destin_state'] = tract_df.GEOID.str[18:20]
tract_df['destin_county'] = tract_df.GEOID.str[20:23]
tract_df['destin_tract'] = tract_df.GEOID.str[23:]
tract_df.head()

,GEOID,TBLID,LINENO,EST,MOE,SOURCE,origin_state,origin_county,origin_tract,destin_state,destin_county,destin_tract
0,C5400US0600140010006001400500,B302201,23,20,+/-30,NaN,06,001,400100,06,001,400500
1,C5400US0600140010006001401000,B302201,16,10,+/-21,NaN,06,001,400100,06,001,401000
2,C5400US0600140010006001401300,B302201,16,10,+/-14,NaN,06,001,400100,06,001,401300
3,C5400US0600140010006001401500,B302201,16,10,+/-14,NaN,06,001,400100,06,001,401500
4,C5400US0600140010006001401700,B302201,16,10,+/-18,NaN,06,001,400100,06,001,401700


In [12]:
### only keep tract to tract flow in the Bay Area
bay_area_county_ids = ['001', '013', '041', '055', '075', '081', '085', '095', '097']
bay_area_tract_flow_df = tract_df.loc[tract_df['origin_county'].isin(bay_area_county_ids) & tract_df['destin_county'].isin(bay_area_county_ids)]
print(tract_df.shape, bay_area_tract_flow_df.shape)
bay_area_tract_flow_df.head()

(618653, 12) (123635, 12)


,GEOID,TBLID,LINENO,EST,MOE,SOURCE,origin_state,origin_county,origin_tract,destin_state,destin_county,destin_tract
0,C5400US0600140010006001400500,B302201,23,20,+/-30,NaN,06,001,400100,06,001,400500
1,C5400US0600140010006001401000,B302201,16,10,+/-21,NaN,06,001,400100,06,001,401000
2,C5400US0600140010006001401300,B302201,16,10,+/-14,NaN,06,001,400100,06,001,401300
3,C5400US0600140010006001401500,B302201,16,10,+/-14,NaN,06,001,400100,06,001,401500
4,C5400US0600140010006001401700,B302201,16,10,+/-18,NaN,06,001,400100,06,001,401700


In [22]:
by_time = pd.pivot_table(bay_area_tract_flow_df, values='EST', index=['origin_state', 'origin_county', 'origin_tract', 'destin_state', 'destin_county', 'destin_tract'], columns=['LINENO'], aggfunc=np.sum, fill_value=0)
by_time = by_time.reset_index().rename(columns={16: 'drive_alone_before_9', 17: 'carpool_before_9', 23: 'drive_alone_after_9', 24: 'carpool_after_9'})
by_time['vehicles'] = by_time['drive_alone_before_9'] + by_time['carpool_before_9']/2 + by_time['drive_alone_after_9'] + by_time['carpool_after_9']/2
display(by_time.head(2))
print(by_time.iloc[:, 6:].sum())
by_time.to_csv('ctpp/ctpp_B302201_bay_area_mode_time.csv')

LINENO,origin_state,origin_county,origin_tract,destin_state,destin_county,destin_tract,drive_alone_before_9,carpool_before_9,drive_alone_after_9,carpool_after_9,vehicles
0,06,001,400100,06,001,400500,0,0,20,0,20.0
1,06,001,400100,06,001,401000,10,0,0,0,10.0


LINENO
drive_alone_before_9    1291837.0
carpool_before_9         209128.0
drive_alone_after_9      611888.0
carpool_after_9           81991.0
vehicles                2049284.5
dtype: float64
